# The Wonderful World of Data Quality Tools in Python
Sam Bail, Data Umbrella, March 2021

## Imports and data loading
We've got some CSV files with 10,000 row samples of NYC taxi ride data from January and February 2019 which I'm loading here.

In [58]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [56]:
csv1 = 'data/yellow_tripdata_sample_2019-01.csv'
df1 = pd.read_csv(csv1)
df1.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,fare_amount
0,2,2019-01-05 06:36:51,2019-01-05 06:50:42,1,14.0
1,1,2019-01-23 15:22:13,2019-01-23 15:32:50,1,12.5
2,2,2019-01-04 10:54:47,2019-01-04 11:18:31,2,17.0
3,1,2019-01-05 12:07:08,2019-01-05 12:14:06,1,6.0
4,2,2019-01-04 18:23:00,2019-01-04 18:25:22,5,3.5


In [57]:
csv2 = 'data/yellow_tripdata_sample_2019-02.csv'
df2 = pd.read_csv(csv2)
df2.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,fare_amount
0,1,2019-02-07 15:48:06,2019-02-07 16:00:40,2,9.0
1,2,2019-02-11 15:19:56,2019-02-11 15:40:10,1,14.0
2,2,2019-02-15 20:03:53,2019-02-15 20:08:34,1,5.0
3,2,2019-02-03 15:16:04,2019-02-03 15:21:49,1,5.5
4,2,2019-02-15 09:23:09,2019-02-15 09:42:51,6,13.5


## Example 1: Pandas "describe" for DataFrames
A simple "profiler" for dataframes. It just gives you some basic statistics on numeric columns.

In [3]:
df1.describe()

,vendor_id,passenger_count,fare_amount
count,10000.000000,10000.00000,10000.000000
mean,1.643800,1.57160,12.485031
std,0.535677,1.19859,32.571797
min,1.000000,1.00000,-52.000000
25%,1.000000,1.00000,6.000000
50%,2.000000,1.00000,9.000000
75%,2.000000,2.00000,13.500000
max,4.000000,6.00000,3004.000000


# Example 2: Pandas Profiling
Like a very sophisticated extension of .describe() on Pandas dataframes, 
creates a more detailed profile report of the data.

**Note:** The key difference between the two dataframes is the minimum in the *passenger_count* column:
* In the January data (df1), we have passenger counts from 1 through 6. 
* In the February data (df2), we have counts from 0 through 6, which looks like a bug.

In [59]:
# Simple way to profile our dataframe and look at the nicely rendered HTML result

from pandas_profiling import ProfileReport
ProfileReport(df1, title="Pandas Profiling Report for df1").to_notebook_iframe()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
ProfileReport(df2, title="Pandas Profiling Report for df2").to_notebook_iframe()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Example 3: TDDA (Test-Driven Data Analysis)
TDDA allows us to generate "constraints" from a reference data asset and verify whether another data asset matches those constraints.

In [49]:
# Generate the constraints based on the January dataframe

from tdda.constraints import discover_df, verify_df
constraints = discover_df(df1)
constraints_path = 'tdda_refs/example_constraints.tdda'
with open(constraints_path, 'w') as f:
    f.write(constraints.to_json())
    
# Show the generated constraints
print(str(constraints))

FIELDS:

Field vendor_id:
           type: TypeConstraint(value='int')
            min: MinConstraint(value=1, precision=None)
            max: MaxConstraint(value=4, precision=None)
           sign: SignConstraint(value='positive')
      max_nulls: MaxNullsConstraint(value=0)

Field pickup_datetime:
           type: TypeConstraint(value='string')
     min_length: MinLengthConstraint(value=19)
     max_length: MaxLengthConstraint(value=19)
      max_nulls: MaxNullsConstraint(value=0)

Field dropoff_datetime:
           type: TypeConstraint(value='string')
     min_length: MinLengthConstraint(value=19)
     max_length: MaxLengthConstraint(value=19)
      max_nulls: MaxNullsConstraint(value=0)

Field passenger_count:
           type: TypeConstraint(value='int')
            min: MinConstraint(value=1, precision=None)
            max: MaxConstraint(value=6, precision=None)
           sign: SignConstraint(value='positive')
      max_nulls: MaxNullsConstraint(value=0)

Field fare_amount:
   

In [46]:
# Verify that the January data matches the constraints - this should match of course!

v1 = verify_df(df1, constraints_path, type_checking='strict', epsilon=0)
print(str(v1))

FIELDS:

vendor_id: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

pickup_datetime: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

dropoff_datetime: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

passenger_count: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

fare_amount: 0 failures  4 passes  type ✓  min ✓  max ✓  max_nulls ✓

SUMMARY:

Constraints passing: 22
Constraints failing: 0


In [62]:
# Verify that the February data matches the constraints - this should fail
# because we have a different min for passenger_count

v2 = verify_df(df2, constraints_path, type_checking='strict', epsilon=0)
print(str(v2))

FIELDS:

vendor_id: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

pickup_datetime: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

dropoff_datetime: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

passenger_count: 2 failures  3 passes  type ✓  min ✗  max ✓  sign ✗  max_nulls ✓

fare_amount: 0 failures  4 passes  type ✓  min ✓  max ✓  max_nulls ✓

SUMMARY:

Constraints passing: 20
Constraints failing: 2


## Example 4: Bulwark
Data testing framework that lets you add tests on methods that return Pandas dataframes. 
Has some built-in tests and allows custom methods for tests. List of all built-in tests ("checks"): https://bulwark.readthedocs.io/en/stable/bulwark.checks.html

In [89]:
import bulwark.decorators as dc

# Option 1: Add checks/assertions as decorators on methods that generate dataframes
# This will return the df if all tests pass, and raise errors if any of the tests fail
@dc.HasNoNans()
@dc.IsShape((10000, 5)) # 10000 rows, 5 columns
@dc.HasValsWithinRange(items={"passenger_count": (1,6)}) # min and max are inclusive here
def load_and_test_csv(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df

# Test this out with the January data
load_and_test_csv(csv1).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,fare_amount
0,2,2019-01-05 06:36:51,2019-01-05 06:50:42,1,14.0
1,1,2019-01-23 15:22:13,2019-01-23 15:32:50,1,12.5
2,2,2019-01-04 10:54:47,2019-01-04 11:18:31,2,17.0
3,1,2019-01-05 12:07:08,2019-01-05 12:14:06,1,6.0
4,2,2019-01-04 18:23:00,2019-01-04 18:25:22,5,3.5


In [84]:
# Now let's test the February data - this should fail the "has vals within range" test
load_and_test_csv(csv2).head()

AssertionError: ('Outside range', 0       False
1       False
2       False
3       False
4       False
        ...  
9995     True
9996     True
9997     True
9998     True
9999     True
Name: passenger_count, Length: 10000, dtype: bool)

In [90]:
import bulwark.checks as ck

# Option 2: You can also use the built-in tests ("checks") directly on a dataframe
ck.is_shape(df2, (10000, 5)).head() # 10000 rows, 5 columns
ck.has_vals_within_range(df2, items={"passenger_count": (1,6)}).head()

AssertionError: ('Outside range', 0       False
1       False
2       False
3       False
4       False
        ...  
9995     True
9996     True
9997     True
9998     True
9999     True
Name: passenger_count, Length: 10000, dtype: bool)

## Example 5: Voluptous/Opulent Pandas
Voluptous is a data validation library that allows you to specify a "schema" to validate JSON/YAML. 
Opulent Pandas is a df-focused “version” of Voluptuous. The syntax to define and validate a schema
is very similar. 

**Note:** Opulent Pandas doesn't look like it's being actively maintained, so I'm
only showing Voluptuous here.

In [150]:
from voluptuous import All, Range, ALLOW_EXTRA

# I had to fiddle a little to pass the right dict-type data structure into the schema
# This returns the df if the tests pass, or throws an error if a test fails 
def validate_df(df):
    schema = Schema(
        {
            'vendor_id': All(int)
            'passenger_count': All(int, Range(min=1, max=6))
        }, 
        extra=ALLOW_EXTRA
    )
    for r in df.to_dict('records'):
        schema(r)
    return df

# Test this out with df1, which should pass
validate_df(df1).head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,fare_amount
0,2,2019-01-05 06:36:51,2019-01-05 06:50:42,1,14.0
1,1,2019-01-23 15:22:13,2019-01-23 15:32:50,1,12.5
2,2,2019-01-04 10:54:47,2019-01-04 11:18:31,2,17.0
3,1,2019-01-05 12:07:08,2019-01-05 12:14:06,1,6.0
4,2,2019-01-04 18:23:00,2019-01-04 18:25:22,5,3.5


In [151]:
validate_df(df2).head()

MultipleInvalid: value must be at least 1 for dictionary value @ data['passenger_count']

In [106]:
df1.to_dict()

{'vendor_id': {0: 2,
  1: 1,
  2: 2,
  3: 1,
  4: 2,
  5: 1,
  6: 2,
  7: 2,
  8: 2,
  9: 1,
  10: 2,
  11: 2,
  12: 1,
  13: 1,
  14: 1,
  15: 1,
  16: 2,
  17: 1,
  18: 2,
  19: 1,
  20: 1,
  21: 1,
  22: 2,
  23: 2,
  24: 2,
  25: 1,
  26: 2,
  27: 1,
  28: 4,
  29: 2,
  30: 2,
  31: 2,
  32: 2,
  33: 1,
  34: 2,
  35: 2,
  36: 1,
  37: 2,
  38: 2,
  39: 2,
  40: 2,
  41: 2,
  42: 1,
  43: 1,
  44: 2,
  45: 2,
  46: 2,
  47: 2,
  48: 2,
  49: 2,
  50: 2,
  51: 1,
  52: 2,
  53: 2,
  54: 1,
  55: 2,
  56: 2,
  57: 2,
  58: 2,
  59: 2,
  60: 2,
  61: 2,
  62: 1,
  63: 2,
  64: 2,
  65: 2,
  66: 1,
  67: 2,
  68: 2,
  69: 2,
  70: 2,
  71: 1,
  72: 2,
  73: 1,
  74: 2,
  75: 2,
  76: 2,
  77: 2,
  78: 1,
  79: 1,
  80: 1,
  81: 2,
  82: 2,
  83: 2,
  84: 2,
  85: 1,
  86: 1,
  87: 1,
  88: 1,
  89: 2,
  90: 2,
  91: 2,
  92: 2,
  93: 2,
  94: 1,
  95: 1,
  96: 2,
  97: 2,
  98: 2,
  99: 2,
  100: 1,
  101: 2,
  102: 1,
  103: 2,
  104: 2,
  105: 2,
  106: 2,
  107: 1,
  108: 1,
  109: 